# stromgedacht api
https://www.stromgedacht.de/

https://www.stromgedacht.de/api-info/

https://api.stromgedacht.de/index.html

https://www.stromgedacht.de/api-docs


In [3]:
import requests
import json
import pprint

In [1]:
# StromGedacht API
#
# Get current state for zip=70173 (Stuttgart)
# # response
# Meaning of states (cf. https://www.stromgedacht.de):
# 1 = green
# 2 = yellow
# 3 = orange
# 4 = red
#

zip_code=70173
stromgedachtURL_now = "https://api.stromgedacht.de/v1/now?zip="
requestURL = stromgedachtURL_now+str(zip_code)
response = requests.get(requestURL, headers = {"accept":"application/json"})
content = json.loads(response.text)
state = content["state"]
print(state)


1


In [10]:
# Be sure to have the requests library installed ('pip install requests')
import requests
import json

# StromGedacht API
#
# Get states for zip=70173 (Stuttgart) from 2023-04-11T00:00:00+02:00 to 2023-04-17T23:59:59+02:00
# Please substitute the start and end dates above with valid dates as the API only provides data for up to four days in the past and two days in the future
#
# Meaning of states (cf. https://www.stromgedacht.de):
# 1 = green
# 2 = yellow
# 3 = orange
# 4 = red
# Die Bedeutung der StromGedacht-Zustände ist – wie aus der StromGedacht-App bekannt – wie folgt:

#     1 = grüner Zustand: Normalbetrieb – Du musst nichts weiter tun
#     2 = gelber Zustand: Verbrauch vorverlegen – Strom jetzt nutzen
#     3 = oranger Zustand: Verbrauch reduzieren, um Kosten und CO2 zu sparen
#     4 = roter Zustand: Verbrauch reduzieren, um Strommangel zu verhindern
#
starttime="2023-05-01T00%3A00%3A00%2B02%3A00"
endtime="2023-05-07T23%3A59%3A59%2B02%3A00"

stromgedachtURL_states = "https://api.stromgedacht.de/v1/states?zip="
requestURL = stromgedachtURL_states+str(zip_code)+"&from="+starttime+"&to="+endtime
# stromgedachtURL = "https://api.stromgedacht.de/v1/states?zip=70173&from=2023-04-11T00%3A00%3A00%2B02%3A00&to=2023-04-17T23%3A59%3A59%2B02%3A00"
print(requestURL)
response = requests.get(requestURL, headers = {"accept":"application/json"})
content = json.loads(response.text)
pprint.pprint(response)
states = content["states"]
print(states)
print(states[0]["from"])
print(states[0]["to"])
print(states[0]["state"])


https://api.stromgedacht.de/v1/states?zip=70173&from=2023-05-01T00%3A00%3A00%2B02%3A00&to=2023-05-07T23%3A59%3A59%2B02%3A00
<Response [200]>
[{'from': '2023-05-01T00:00:00+02:00', 'to': '2023-05-07T23:59:59+02:00', 'state': 1}]
2023-05-01T00:00:00+02:00
2023-05-07T23:59:59+02:00
1
